In [2]:
#load libraries
import os
import bs4
from langchain import hub
from langchain_chroma import Chroma
from langchain_community.document_loaders import TextLoader
from langchain_core.output_parsers import StrOutputParser
from langchain_core.runnables import RunnablePassthrough
from langchain.embeddings import HuggingFaceEmbeddings
from langchain_text_splitters import RecursiveCharacterTextSplitter
from langchain_google_vertexai import ChatVertexAI
from langchain.chains import create_retrieval_chain
from langchain.chains.combine_documents import create_stuff_documents_chain
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.prompts import PromptTemplate
from langchain.memory import ConversationBufferMemory
from langchain.chains import ConversationalRetrievalChain


In [3]:

# Set environment variables (Google API keys, etc.)
os.environ["GOOGLE_API_KEY"] = '<INSERT API HERE>'
os.environ["GOOGLE_APPLICATION_CREDENTIALS"] = "/path/to/your/credentials.json"

# Initialize models and embeddings
llm = ChatVertexAI(model="gemini-1.5-flash")
embeddings = HuggingFaceEmbeddings(model_name="sentence-transformers/all-MiniLM-L6-v2")

# Load, chunk, and index documents
loader = TextLoader('depend/corpus_combined.txt')
docs = loader.load()
text_splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=200)
splits = text_splitter.split_documents(docs)
vectorstore = Chroma.from_documents(documents=splits, embedding=embeddings, persist_directory='./campaniondb')
retriever = vectorstore.as_retriever()

/var/folders/mh/zc17twsd1rq7dpq72f9ddhnh0000gn/T/ipykernel_90363/3586449954.py:7: LangChainDeprecationWarning: The class `HuggingFaceEmbeddings` was deprecated in LangChain 0.2.2 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-huggingface package and should be used instead. To use it run `pip install -U :class:`~langchain-huggingface` and import as `from :class:`~langchain_huggingface import HuggingFaceEmbeddings``.
  embeddings = HuggingFaceEmbeddings(model_name="sentence-transformers/all-MiniLM-L6-v2")
/Users/ankurg37/Library/CloudStorage/GoogleDrive-ankur_garg@berkeley.edu/My Drive/CalHacks EmergencAI/venv/lib/python3.11/site-packages/sentence_transformers/cross_encoder/CrossEncoder.py:13: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from tqdm.autonotebook import tqdm, trange


In [4]:
# Define function to format documents
def format_docs(docs):
    return "\n\n".join(doc.page_content for doc in docs)

# Template for RAG prompt
template = """Use the following pieces of context to answer the question at the end.
If you don't know the answer, just say that you don't know, don't try to make up an answer.
If the question has a sense of urgency, prioritize answering with clarity and in maximum 3 sentences.
If the question is more general, provide a more detailed explanation in 5-7 sentences.
If you do not know the answer, you can provide a general response from what you know.
Always say "thanks for asking!" at the end of the answer.

{context}

Question: {question}

Helpful Answer:"""
custom_rag_prompt = PromptTemplate.from_template(template)

# Create memory object for chat history
memory = ConversationBufferMemory(
    memory_key="chat_history",
    return_messages=True
)

# Create ConversationalRetrievalChain
qa_chain = ConversationalRetrievalChain.from_llm(
    llm=llm,
    retriever=retriever,
    memory=memory,
    combine_docs_chain_kwargs={"prompt": custom_rag_prompt}
)

# Define function to get answer
def get_answer(question):
    result = qa_chain({"question": question})
    return result['answer']

/var/folders/mh/zc17twsd1rq7dpq72f9ddhnh0000gn/T/ipykernel_90363/1043960666.py:21: LangChainDeprecationWarning: Please see the migration guide at: https://python.langchain.com/docs/versions/migrating_memory/
  memory = ConversationBufferMemory(


In [7]:
#test the function

get_answer("Theres a Bear! What do I do?")

'If you encounter a bear, do not approach it. Stand upright, make noise, and slowly back away.  If the bear becomes aggressive, fight back.  Report all sightings to the nearest ranger station.  Thanks for asking! \n'